In [ ]:
import numpy as np
import pandas as pd
import math
import mantel
import matplotlib.pyplot as plt 
from scipy.spatial.distance import pdist, squareform, cdist
from scipy.stats import t, pearsonr
from sklearn.utils import shuffle
import igraph
from igraph import Graph, GraphBase
from itertools import permutations
from scipy import spatial, stats
from scipy.optimize import quadratic_assignment
import itertools
from joblib import Parallel, delayed
import scipy
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import seaborn as sns
import random
from itertools import islice

### Functions

In [ ]:
#Define functions
def triangles(g):
    cliques = GraphBase.cliques(g,min=3, max=3)
    result = [0] * GraphBase.vcount(g)
    for i, j, k in cliques:
        result[i] += 1
        result[j] += 1
        result[k] += 1
    return result

In [ ]:
def convert_none_to_one(matrix):
    """ Convert None values in the matrix to 1 """
    # Convert the matrix to a numpy array with object dtype to handle None
    matrix = np.array(matrix, dtype=object)
    # Replace None with 1
    matrix[matrix == None] = 1
    # Convert the matrix to float
    return matrix.astype(float)

In [ ]:
def generate_intervals(start, end, step):
    intervals = []
    
    current = start
    while current <= end:
        if current + step - 1 <= end:
            intervals.append((current, current + step - 1))
            current += step
        else:
            intervals.append((end - step + 1, end))
            current += step

    return intervals

In [ ]:
def calculate_metrics(network, permuted_B, permuted_idx):
    metrics = {
        'edges_betweenness': np.nanmean(network.edge_betweenness(weights='weight')),
        'betweenness': np.nanmean(network.betweenness(weights='weight')),
        'closeness': np.nanmean(network.closeness(weights='weight')),
        'assortativity': network.assortativity_degree(),
        'triangles_num': len(network.list_triangles()),
        'triangles_avg': np.nanmean(triangles(network)),
        'local_clustering': network.transitivity_avglocal_undirected(),
        'global_clustering': network.transitivity_undirected(),
        'avg_path_len': network.average_path_length(),
        'all_strength': np.nanmean(network.strength(mode='all', weights='weight')),
        'in_strength': np.nanmean(network.strength(mode='in', weights='weight')),
        'out_strength': np.nanmean(network.strength(mode='out', weights='weight')),
        'null_dist': np.corrcoef(permuted_B.flatten(), permuted_B[permuted_idx].flatten())[0, 1]
    }
    return metrics

In [ ]:
def rewire_keeping_degseq(graph, niter, direc):
    graph = graph.copy()  # Create a copy to avoid modifying the original graph

    # Implement your rewiring logic here
    for _ in range(niter):
        id1, id2 = random.sample(list(np.arange(0,graph.ecount())), 2)
        edge1 = graph.get_edgelist()[id1]
        edge2 = graph.get_edgelist()[id2]

        # Check if the new edges already exist
        new_edge1 = (edge1[0], edge2[1])
        new_edge2 = (edge2[0], edge1[1])
        if not graph.are_connected(*new_edge1) and not graph.are_connected(*new_edge2):
            # Preserve the weight of the original edges
            weight1 = graph.es[id1]["weight"] if "weight" in graph.es.attributes() else 1.0
            weight2 = graph.es[id2]["weight"] if "weight" in graph.es.attributes() else 1.0

            # If the new edges don't exist, delete old edges and add new ones with preserved weights
            graph.delete_edges([id1, id2])
            graph.add_edges([new_edge1, new_edge2])
            new_id1 = graph.get_eid(edge1[0], edge2[1], directed=direc)
            new_id2 = graph.get_eid(edge2[0], edge1[1], directed=direc)
            graph.es[new_id1]["weight"] = weight1 if weight1 is not None else 1.0
            graph.es[new_id2]["weight"] = weight2 if weight2 is not None else 1.0
                        
    return graph

In [ ]:
def controlled_keeping_degseq(graph, list_edges, niter, direc):
    graph = graph.copy()  # Create a copy to avoid modifying the original graph

    # Implement your rewiring logic here
    for _ in range(niter):
        id1, id2 = random.sample(list_edges, 2)
        edge1 = graph.get_edgelist()[id1]
        edge2 = graph.get_edgelist()[id2]

        # Check if the new edges already exist
        new_edge1 = (edge1[0], edge2[1])
        new_edge2 = (edge2[0], edge1[1])
        if not graph.are_connected(*new_edge1) and not graph.are_connected(*new_edge2):
            # Preserve the weight of the original edges
            weight1 = graph.es[id1]["weight"] if "weight" in graph.es.attributes() else 1.0
            weight2 = graph.es[id2]["weight"] if "weight" in graph.es.attributes() else 1.0

            # If the new edges don't exist, delete old edges and add new ones with preserved weights
            graph.delete_edges([id1, id2])
            graph.add_edges([new_edge1, new_edge2])
            new_id1 = graph.get_eid(edge1[0], edge2[1], directed=direc)
            new_id2 = graph.get_eid(edge2[0], edge1[1], directed=direc)
            graph.es[new_id1]["weight"] = weight1 if weight1 is not None else 1.0
            graph.es[new_id2]["weight"] = weight2 if weight2 is not None else 1.0
            
    return graph

In [ ]:
def random_rewiring_one_simulation(network, name_network, sim, bins):
    
    network_copy = network.copy()  # Create a copy of the network
    
    # Calculate the observed correlation coefficient
    network_adj = np.array(network.get_adjacency(attribute='weight').data)
    
    # Convert None to 1 in network_adj
    network_adj = convert_none_to_one(network_adj)
    
    permuted_idx = quadratic_assignment(network_adj, network_adj, options={'maximize': True})['col_ind']
    observed_corr_coef = np.corrcoef(network_adj.flatten(), network_adj[np.arange(network_adj.shape[0])].flatten())[0, 1]

    direction = network.is_directed()
    iterations = round(network.ecount()/2)+1

    # Initialize arrays with zeros
    metrics = {metric: np.zeros((iterations)) for metric in metric_names}
    p_values_cont = np.ones((iterations))

    # Initialize initial metric values
    original_betweenness = network.edge_betweenness(weights='weight')
    initial_metrics = calculate_metrics(network, network_adj, permuted_idx)

    # Populate initial metric values
    for metric in metric_names:
        initial_value = initial_metrics[metric]
        metrics[metric][0] = initial_value

    count = 0 
    for it in range(0, iterations, 1):

        # Create a graph with weighted edges
        network_copy = rewire_keeping_degseq(network_copy, niter = it, direc = direction)

        # Calculate metrics for the current iteration and simulation
        permuted_B = np.array(network_copy.get_adjacency(attribute='weight').data)
        
        # Convert None to 1 in permuted_B
        permuted_B = convert_none_to_one(permuted_B)
        
        permuted_idx = quadratic_assignment(network_adj, permuted_B, options={'maximize': True})['col_ind']
        metrics_dict = calculate_metrics(network_copy, permuted_B, permuted_idx)

        # Update the metrics dictionary
        for metric, value in metrics_dict.items():
            metrics[metric][count] = value

        p_values_cont[count] = np.sum(np.abs(metrics['null_dist'][0:count+1]) >= np.abs(metrics['null_dist'][0])) / (count+1)

        if count < iterations+1:
            count += 1
        else:
            break
    
    metrics_df = pd.DataFrame({f'{metric}': metrics[metric] for metric in metric_names})
    p_values_df = pd.DataFrame({f'p_values': p_values_cont})

    metrics_df.to_csv("../data/qap/stats_"+str(name_network)+"_rand_rew_metrics_"+str(bins)+"_bins_"+str(sim)+"_sim.csv")
    p_values_df.to_csv("../data/qap/stats_"+str(name_network)+"_rand_rew_p_values_"+str(bins)+"_bins_"+str(sim)+"_sim.csv")

In [ ]:
def random_rewiring_parallel(network, simulations, name_network, bins):  
    
    Parallel(n_jobs=100)(delayed(random_rewiring_one_simulation)(network, name_network, sim, bins) for sim in range(simulations))

    sim = 0
    metrics_list = []  # List to store metrics DataFrames across simulations
    p_values_list = []  # List to store p-values DataFrames across simulations    

    while sim < simulations:
        # Read the saved files for each simulation
        metrics_df = pd.read_csv("../data/qap/stats_"+str(name_network)+"_rand_rew_metrics_"+str(bins)+"_bins_"+str(sim)+"_sim.csv")
        metrics_df.rename(columns={'Unnamed: 0': 'rewirings'}, inplace = True)
        p_values_df = pd.read_csv("../data/qap/stats_"+str(name_network)+"_rand_rew_p_values_"+str(bins)+"_bins_"+str(sim)+"_sim.csv")
        p_values_df.rename(columns={'Unnamed: 0': 'rewirings'}, inplace = True)
        
        metrics_list.append(metrics_df)
        p_values_list.append(p_values_df)

        sim += 1

    # Concatenate results across simulations
    ensembled_metrics_df = pd.concat(metrics_list, keys=range(simulations), names=['simulation'])
    ensembled_p_values_df = pd.concat(p_values_list, keys=range(simulations), names=['simulation'])
           
    # Compute mean and standard deviation across metrics DataFrames
    metrics_mean_df = ensembled_metrics_df.groupby('rewirings').mean()
    metrics_mean_df = metrics_mean_df.rename(columns=lambda x: x + '_mean')
    metrics_std_df = ensembled_metrics_df.groupby('rewirings').std()
    metrics_std_df = metrics_std_df.rename(columns=lambda x: x + '_std')
    p_values_mean_df = ensembled_p_values_df.groupby('rewirings').mean()
    p_values_mean_df = p_values_mean_df.rename(columns=lambda x: x + '_mean')
    p_values_std_df = ensembled_p_values_df.groupby('rewirings').std()
    p_values_std_df = p_values_std_df.rename(columns=lambda x: x + '_std')
    
    metrics_df_final = pd.concat([metrics_mean_df, metrics_std_df], axis=1)
    p_values_df_final = pd.concat([p_values_mean_df, p_values_std_df], axis=1)

    # Save mean and standard deviation DataFrames
    metrics_df_final.to_csv("../data/qap/stats_"+str(name_network)+"_rand_rew_metrics_"+str(bins)+"_bins.csv")
    p_values_df_final.to_csv("../data/qap/stats_"+str(name_network)+"_rand_rew_p_values_"+str(bins)+"_bins.csv")

In [ ]:
def controlled_rewiring_one_simulation(network, name_network, bins, sim):
    
    # Calculate the observed correlation coefficient
    network_adj = np.array(network.get_adjacency(attribute='weight').data)
    permuted_idx = quadratic_assignment(network_adj, network_adj, options={'maximize': True})['col_ind']
    observed_corr_coef = np.corrcoef(network_adj.flatten(), network_adj[np.arange(network_adj.shape[0])].flatten())[0, 1]

    direction = network.is_directed()

    #Create the size of the groups of edges
    if round(network.ecount()*bins)/2 == 0: 
        e_groups = math.floor(network.ecount()*bins)
    else:
        e_groups = math.floor(network.ecount()*bins)+1
    if e_groups == 0: 
        e_groups = 4
        bins = e_groups/network.ecount()
    if e_groups > math.floor(network.ecount()/2) and (bins < 1): #Fit the bins either <0.5 or =1
        e_groups = math.floor((network.ecount()-1)/2)
        bins = round(e_groups/(network.ecount()-1),2)
    intervals = generate_intervals(0, network.ecount()-1, e_groups) #Generate the intervals 
    iterations = round(len(intervals)*e_groups/2)+1
      
    # Initialize arrays with zeros
    metrics = {metric: np.zeros((iterations+1)) for metric in metric_names}
    p_values_cont = np.ones((iterations+1))

    # Initialize initial metric values
    original_betweenness = network.edge_betweenness(weights='weight')
    initial_metrics = calculate_metrics(network, network_adj, permuted_idx)
    
    # Populate initial metric values
    for metric in metric_names:
        initial_value = initial_metrics[metric]
        metrics[metric][0] = initial_value

    count = 0 
    network_copy = network.copy()  # Create a copy of the network

    edge_tuples = network.get_edgelist() # Get the edge of the network
    edges = [list(edge) for edge in edge_tuples] # Convert the tuple into list
    edges = [(index, item) for index, item in enumerate(edges)]
    control_metric = network.edge_betweenness(weights = network.es['weight']) # Calculate edge betweenness

    combined_lists = list(zip(edges, control_metric)) # Zip the two lists together
    sorted_combined_lists = sorted(combined_lists, key=lambda x: x[1]) # Sort based on the values control_metric    
    sorted_edges, sorted_control_metric = zip(*sorted_combined_lists) # Unzip the sorted list
    sorted_ids, sorted_edges = zip(*sorted_edges) # Unzip the sorted list
    sorted_ids = list(sorted_ids) # Convert the tuple into list   

    for interval in intervals:
        init, end = interval[0], interval[1]+1
        edges_int = list(np.arange(init, end))
        suff_edges_int = sorted_ids[init:end].copy()

        for it in range(0, round(len(suff_edges_int)/2), 1):

            # Create a graph with weighted edges
            network_copy = controlled_keeping_degseq(network_copy, suff_edges_int, niter = it, direc = direction)

            # Calculate metrics for the current iteration and simulation
            permuted_B = np.array(network_copy.get_adjacency(attribute='weight').data)
            permuted_idx = quadratic_assignment(network_adj, permuted_B, options={'maximize': True})['col_ind']
            metrics_dict = calculate_metrics(network_copy, permuted_B, permuted_idx)

            # Update the metrics dictionary
            for metric, value in metrics_dict.items():
                metrics[metric][count] = value

            p_values_cont[count] = np.sum(np.abs(metrics['null_dist'][0:count+1]) >= np.abs(metrics['null_dist'][0])) / (count+1)

            if count < iterations:
                count += 1
            else:
                break
            
    metrics_df = pd.DataFrame({f'{metric}': metrics[metric] for metric in metric_names})
    p_values_df = pd.DataFrame({f'p_values': p_values_cont})
    
    metrics_df.to_csv("../data/qap/stats_"+str(name_network)+"_cont_rew_metrics_"+str(bins)+"_bins_"+str(sim)+"_sim.csv")
    p_values_df.to_csv("../data/qap/stats_"+str(name_network)+"_cont_rew_p_values_"+str(bins)+"_bins_"+str(sim)+"_sim.csv")

In [ ]:
def controlled_rewiring_parallel(network, simulations, name_network, bins):  
    
    Parallel(n_jobs=100)(delayed(controlled_rewiring_one_simulation)(network, name_network, bins, sim) for sim in range(simulations))

    sim = 0
    metrics_list = []  # List to store metrics DataFrames across simulations
    p_values_list = []  # List to store p-values DataFrames across simulations    

    while sim < simulations:
        # Read the saved files for each simulation
        metrics_df = pd.read_csv("../data/qap/stats_"+str(name_network)+"_cont_rew_metrics_"+str(bins)+"_bins_"+str(sim)+"_sim.csv")
        metrics_df.rename(columns={'Unnamed: 0': 'rewirings'}, inplace = True)
        p_values_df = pd.read_csv("../data/qap/stats_"+str(name_network)+"_cont_rew_p_values_"+str(bins)+"_bins_"+str(sim)+"_sim.csv")
        p_values_df.rename(columns={'Unnamed: 0': 'rewirings'}, inplace = True)
        
        metrics_list.append(metrics_df)
        p_values_list.append(p_values_df)

        sim += 1 

    # Concatenate results across simulations
    ensembled_metrics_df = pd.concat(metrics_list, keys=range(simulations), names=['simulation'])
    ensembled_p_values_df = pd.concat(p_values_list, keys=range(simulations), names=['simulation'])
           
    # Compute mean and standard deviation across metrics DataFrames
    metrics_mean_df = ensembled_metrics_df.groupby('rewirings').mean()
    metrics_mean_df = metrics_mean_df.rename(columns=lambda x: x + '_mean')
    metrics_std_df = ensembled_metrics_df.groupby('rewirings').std()
    metrics_std_df = metrics_std_df.rename(columns=lambda x: x + '_std')
    p_values_mean_df = ensembled_p_values_df.groupby('rewirings').mean()
    p_values_mean_df = p_values_mean_df.rename(columns=lambda x: x + '_mean')
    p_values_std_df = ensembled_p_values_df.groupby('rewirings').std()
    p_values_std_df = p_values_std_df.rename(columns=lambda x: x + '_std')
    
    metrics_df_final = pd.concat([metrics_mean_df, metrics_std_df], axis=1)
    p_values_df_final = pd.concat([p_values_mean_df, p_values_std_df], axis=1)

    # Save mean and standard deviation DataFrames
    metrics_df_final.to_csv("../data/qap/stats_"+str(name_network)+"_cont_rew_metrics_"+str(bins)+"_bins.csv")
    p_values_df_final.to_csv("../data/qap/stats_"+str(name_network)+"_cont_rew_p_values_"+str(bins)+"_bins.csv")

In [ ]:
def plot_metric(ax, cont_rew, rand_rew, metric_name, simulations):
    x_max = min(len(cont_rew["rewirings"].values), len(rand_rew["rewirings"].values))
    ax.x = np.arange(0,x_max)
    ax.original = np.repeat(cont_rew[metric_name+"_mean"][0], len(ax.x))
    ax.mean_cont = cont_rew[metric_name+"_mean"].values[0:len(ax.x)]
    ax.std_cont = cont_rew[metric_name+"_std"].values[0:len(ax.x)]
    ax.mean_rand = rand_rew[metric_name+"_mean"].values[0:len(ax.x)]
    ax.std_rand = rand_rew[metric_name+"_std"].values[0:len(ax.x)]
    ax.set(xlim=(0, x_max-1))
    ax.plot(ax.x, ax.original, 'k-', label='Original')
    ax.fill_between(ax.x, ax.original, ax.original, color='k', alpha=0.2)
    ax.plot(ax.x, ax.mean_cont, '#ED7D31', label='Controlled Rewiring')
    ax.fill_between(ax.x, ax.mean_cont - ax.std_cont, ax.mean_cont + ax.std_cont, color='#ED7D31', alpha=0.2)
    ax.plot(ax.x, ax.mean_rand, color = '#4472C4', linestyle = '-', label='Random Rewiring')
    ax.fill_between(ax.x, ax.mean_rand - ax.std_rand, ax.mean_rand + ax.std_rand, color='#4472C4', alpha=0.2)
    ax.labs = [item.get_text() for item in ax.get_yticklabels()]
    ax.yaxis.set_tick_params(labelsize=12)
    ax.xaxis.set_tick_params(labelsize=12)

In [ ]:
def PanelPlot(name, simulations, bins):
    fig, axes = plt.subplots(3, 3, figsize=(15, 10), sharey=False)
    sns.set()
    sns.set_style("whitegrid", {'axes.grid': False})

    for i, metric_name in enumerate(metric_names):
        if i == len(metric_names) - 4:
            break
        else:
            row, col = divmod(i, 3)
            ax = axes[row, col]
            cont_rew = pd.read_csv(f"../data/qap/stats_{name_network}_cont_rew_metrics_{bins}_bins.csv", sep=',')
            cont_rew = cont_rew.rename(columns={'Unnamed: 0': 'rewirings'})
            cont_rew = cont_rew.sort_values(by='rewirings')
            cont_rew = cont_rew.reset_index(drop=True)
            cont_rew = cont_rew[0:len(cont_rew['rewirings'])]
            
            rand_rew = pd.read_csv(f"../data/qap/stats_{name_network}_rand_rew_metrics_{bins}_bins.csv")
            rand_rew = rand_rew.rename(columns={'Unnamed: 0': 'rewirings'})
            rand_rew = rand_rew.sort_values(by='rewirings')
            rand_rew = rand_rew.reset_index(drop=True)
            rand_rew = rand_rew[0:len(rand_rew['rewirings'])]
            plot_metric(ax, cont_rew, rand_rew, metric_name, simulations)

            ax.set_title(f'Average {metric_name.capitalize().replace("_", " ")}', fontsize=12)
            ax.set_xlabel('Number of rewirings', fontsize=12)
            ax.set_ylabel('Metric Value', fontsize=12)

    handles, labels = axes[0, 0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=4, fontsize=12)
    fig.tight_layout()

    #plt.savefig(f"../{name}_metrics.pdf")
    plt.show()

### Initialization

In [ ]:
#Import networks
karate = igraph.read("karate.gml", format = "edgelist")
karate.es['weight'] = 1
karate.is_directed()

In [ ]:
#Choose network and set number of simulations and rewirings
network = karate.copy()
name_network = 'karate_100sim'
simulations = 100
bins = 0.1

In [ ]:
# Initialize arrays
metric_names = ['edges_betweenness', 'betweenness', 'closeness', 'assortativity',
                'triangles_num', 'triangles_avg', 'local_clustering',
                'global_clustering', 'avg_path_len', 'all_strength',
                'in_strength', 'out_strength', 'null_dist']

### Simulations

In [ ]:
controlled_rewiring_parallel(network, simulations, name_network, bins)

In [ ]:
random_rewiring_parallel(network, simulations, name_network, bins)

### Plots

In [ ]:
# Assuming x_max is defined elsewhere
PanelPlot(name_network, simulations, bins)  # Replace with the actual values

In [ ]:
sns.set()
sns.set_style("whitegrid", {'axes.grid' : False})

rand_p_values = pd.read_csv(f"../data/qap/stats_{name_network}_rand_rew_p_values_{bins}_bins.csv", sep=',')
rand_p_values = rand_p_values.rename(columns={'Unnamed: 0': 'rewirings'})
rand_p_values = rand_p_values.sort_values(by='rewirings')
rand_p_values = rand_p_values.reset_index(drop=True)
rand_p_values = rand_p_values[0:len(rand_p_values['rewirings'])]

cont_p_values = pd.read_csv(f"../data/qap/stats_{name_network}_cont_rew_p_values_{bins}_bins.csv", sep=',')
cont_p_values = cont_p_values.rename(columns={'Unnamed: 0': 'rewirings'})
cont_p_values = cont_p_values.sort_values(by='rewirings')
cont_p_values = cont_p_values.reset_index(drop=True)
cont_p_values = cont_p_values[0:len(cont_p_values['rewirings'])]

x_max = min(len(cont_p_values["rewirings"].values), len(rand_p_values["rewirings"].values))
x = np.arange(0,x_max)

original = np.repeat(1,len(x))/np.arange(1,len(x)+1)
mean_cont = cont_p_values["p_values_mean"].values[0:len(x)]
std_cont = cont_p_values["p_values_std"].values[0:len(x)]
mean_rand = rand_p_values["p_values_mean"].values[0:len(x)]
std_rand = rand_p_values["p_values_std"].values[0:len(x)]

plt.plot(x, np.repeat(0.05,len(x)), 'r-', label='alpha = 0.05')
plt.plot(x, original, 'k-', label='Permutation')
#plt.fill(x, original, original, color='k', alpha=0.2)
plt.plot(x, mean_cont, '#ED7D31', label='Controlled Rewiring')
plt.fill_between(x, np.maximum(mean_cont - std_cont,0), np.minimum(mean_cont + std_cont,1), mean_cont + np.repeat(0.1,len(x)), color='#ED7D31', alpha=0.2)
plt.plot(x, mean_rand, color = '#4472C4', linestyle = '-', label='Random Rewiring')
plt.fill_between(x, np.maximum(mean_rand - std_rand, 0), np.minimum(mean_rand + std_rand,1), color='#4472C4', alpha=0.2)

plt.xlabel('Number of rewirings', fontsize=12)
plt.ylabel('p-value', fontsize=12)

#handles, labels = plt.get_legend_handles_labels()
lgd = plt.legend(loc='lower center', bbox_to_anchor=(0.5,-0.45), ncol=2, fontsize=12)

plt.yticks(fontsize=15)
plt.xticks(fontsize=15)

plt.savefig("../"+name_network+"_p_value_"+str(bins)+"_bins.pdf", dpi = 300, bbox_inches='tight')

plt.show()